<a href="https://colab.research.google.com/github/minnji88/NLP-study/blob/main/1_1_NeuralNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy 
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

## make_blobs
가우시안 정규분포를 이용해 가상 데이터를 생성한다

인수:
- n_samples : 표본 데이터의 수, 디폴트 100
- n_features : 독립 변수의 수, 디폴트 20
- centers : 생성할 클러스터의 수 혹은 중심, [n_centers, n_features] 크기의 배열. 디폴트 3
- cluster_std: 클러스터의 표준 편차, 디폴트 1.0
- center_box: 생성할 클러스터의 바운딩 박스(bounding box), 디폴트 (-10.0, 10.0))

반환값:
X : [n_samples, n_features] 크기의 배열

독립 변수
y : [n_samples] 크기의 배열


In [ ]:
# 데이터 셋을 만들어 주기(2차원 벡터 형태)
n_dim = 2 # 2차원을 의미
x_train, y_train = make_blobs(n_samples=80, n_features=n_dim,
                              centers=[[0,0],[1,1],[1,0],[0,1]],
                              shuffle=True, cluster_std=0.15)

# 4개의 클러스터가 생성되어 x_train 과 x_test 속 모든 데이터는 0, 1, 2, 3으로 인덱싱
x_test, y_test = make_blobs(n_samples=20, n_features=n_dim,
                            centers=[[0,0],[1,1],[1,0],[0,1]],
                            shuffle=True, cluster_std=0.15)

In [ ]:
x_train.shape

In [ ]:
y_train

In [ ]:
x_test.shape

In [ ]:
y_test

In [ ]:
# 학습 시킬 신경망이 2가지 레이블만 예측하는 기본 모델이므로
# 4개의 레이블을 2개로 합침
def label_map(y_, from_, to_):
    y = numpy.copy(y_)
    for f in from_:
        y[y_ == f] = to_
    return y

y_train = label_map(y_train, [0, 1], 0)
y_train = label_map(y_train, [2, 3], 1)
y_test = label_map(y_test, [0, 1], 0)
y_test = label_map(y_test, [2, 3], 1)

In [ ]:
def vis_data(x,y = None, c = 'r'):
  if y is None:
    y = [None] * len(x)
  for x_, y_ in zip(x, y):
    if y_ is None:
      plt.plot(x_[0], x_[1],markerfacecolor='none', markeredgecolor=c)
    else:
      plt.plot(x_[0], x_[1], c+'o' if y_ == 0 else c+'+')

plt.figure()
vis_data(x_train, y_train, c = 'r') # c를 변경함으로 색 변경 가능 
plt.show()

In [ ]:
# 넘파이 벡터 형식 데이터를 파이토치 텐서로 변형
x_train = torch.FloatTensor(x_train)
print(x_train.shape)
x_test = torch.FloatTensor(x_test)
print(x_test.shape)

In [ ]:

y_train = torch.FloatTensor(y_train)
print(y_train.shape)
y_test = torch.FloatTensor(y_test)
print(y_test.shape)

In [ ]:
class NeuralNet(torch.nn.Module):
        def __init__(self, input_size, hidden_size):
            super(NeuralNet, self).__init__()
            # input_size : 신경망에 입력되는 데이터의 차원 
            self.input_size = input_size
            self.hidden_size  = hidden_size

            self.linear_1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.relu = torch.nn.ReLU()
            self.linear_2 = torch.nn.Linear(self.hidden_size, 1)
            self.sigmoid = torch.nn.Sigmoid()
        
        def forward(self, input_tensor):
            # 입력데이터에 [input_size, hidden_size] 크기의 가중치를 곱하고 편향 더해서
            # [1, hidden_size] 텐서 반환
            linear1 = self.linear_1(input_tensor)
            # 입력값이 0보다 작으면 0, 0보다 크면 입력 값 반환
            relu = self.relu(linear1)
            # 행렬곱을 거쳐서 [1,1] 반환
            linear2 = self.linear_2(relu)
            # 0~ 1사이의 값을 반환
            # 분류 시 0과 1중 어디에 가까운지 알 수 있음.
            output = self.sigmoid(linear2)
            return output

In [ ]:
# 신경망 객체 생성
model = NeuralNet(2, 5)
learning_rate = 0.03 # 학습률 설정
# 오차함수 준비
criterion = torch.nn.BCELoss()
epochs = 2000
# 가중치를 학습률 만큼 갱신
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [ ]:
# 학습되지 않은 모델 성능 TEST
model.eval()
test_loss_before = criterion(model(x_test).squeeze(), y_test)
print('Before Training, test loss is {}'.format(test_loss_before.item()))

In [ ]:
for epoch in range(epochs):
    model.train()
    # 경사값 0으로 초기화
    optimizer.zero_grad()
    train_output = model(x_train)
    train_loss = criterion(train_output.squeeze(), y_train)
    if epoch % 100 == 0:
        print('Train loss at {} is {}'.format(epoch, train_loss.item()))
    # 가중치를 미분해서 방향 구함
    train_loss.backward()
    # 학습률 만큼 이동
    optimizer.step()

In [ ]:
model.eval()
test_loss = criterion(model(x_test).squeeze(), y_test) 
print('After Training, test loss is {}'.format(test_loss.item()))

In [ ]:
torch.save(model.state_dict(), './model.pt')
print('state_dict format of the model: {}'.format(model.state_dict()))

In [ ]:
new_model = NeuralNet(2, 5)
new_model.load_state_dict(torch.load('./model.pt'))
new_model.eval()
print('벡터 [0, 1]이 레이블 1을 가질 확률은 {}'.format\
      (new_model(torch.FloatTensor([0,1])).item()))